In [ ]:
import os
import numpy as np
from skimage import io
import glob
import open3d as o3d
import scipy.io
from tqdm import tqdm
import tifffile as tiff

## Load clustering map data and boundary mask .txt files

In [ ]:
colormap_path = 'colormap.txt'
boundary_mask_path =  'boundary_mask.txt'

colormap = np.loadtxt(colormap_path)
boundary_mask = np.loadtxt(boundary_mask_path)
io.imshow(colormap)

## Visualize diffraction pattern in point cloud format

In [ ]:
diffraction_pattern = 'diffraction_pattern.ply'

pcd = o3d.io.read_point_cloud(diffraction_pattern)
points = np.asarray(pcd.points)

px = points[:,0]
py = points[:,1]
pz = points[:,2]
o3d.visualization.draw_geometries([pcd])

## Get averaged diffraction pattern of domain and boundary

In [ ]:
folder_path = '../mat/'
mat_path = "./mat/diffraction_pattern.mat"
diffraction_pattern_mat = np.asarray(scipy.io.loadmat(mat_path)['data_frame'])

avg_diff_pattern_domain = np.zeros_like(diffraction_pattern_mat)
avg_diff_pattern_boundary = np.zeros_like(diffraction_pattern_mat)

mat_files = glob.glob(os.path.join(folder_path, '*.mat'))

for mat_file in tqdm(mat_files):
    base_name = os.path.basename(mat_file)
    tokens = base_name.split('_')
    xidx = int(tokens[1])
    yidx = int(tokens[2].rsplit('.', 1)[0])
    mat_temp = np.asarray(scipy.io.loadmat(os.path.join(folder_path,base_name))['data_frame'])

    if boundary_mask[xidx, yidx] == 0:
        avg_diff_pattern_domain += mat_temp

    if boundary_mask[xidx, yidx] == 1:
        avg_diff_pattern_boundary += mat_temp

avg_diff_pattern_domain /= np.count_nonzero(boundary_mask == 0)
avg_diff_pattern_boundary /= np.count_nonzero(boundary_mask == 1)

tiff.imwrite("./average_domain_diffraction_pattern.tif", avg_diff_pattern_domain.transpose(2, 1, 0))
tiff.imwrite("./average_boundary_diffraction_pattern.tif", avg_diff_pattern_boundary.transpose(2, 1, 0))

## Get averaged pattern of every cluster

In [ ]:
folder_path = '../mat/'
dim_x, dim_y, dim_z = diffraction_pattern_mat.shape

roi_num = int(np.max(colormap))

avg_diff_pattern_roi = np.zeros((dim_x, dim_y, dim_z, roi_num), dtype=float)
mat_files = glob.glob(os.path.join(folder_path, '*.mat'))

for mat_file in tqdm(mat_files, desc=".mat file reading: "):
    base_name = os.path.basename(mat_file)
    tokens = base_name.split('_')
    xidx = int(tokens[1])
    yidx = int(tokens[2].rsplit('.', 1)[0])
    mat_temp = np.asarray(scipy.io.loadmat(os.path.join(folder_path, base_name))['data_frame'])
    for i in range(roi_num):
        if colormap[xidx, yidx] == i + 1:
            avg_diff_pattern_roi[:,:,:,i] += mat_temp
        else:
            continue

for i in tqdm(range(roi_num), desc="Image writting: "):
    if np.count_nonzero(colormap == i + 1) != 0:
        avg_diff_pattern_roi[:,:,:,i] /= np.count_nonzero(colormap == i + 1)
        tiff.imwrite("./average_diffraction_pattern_" + str(i+1) + ".tif", avg_diff_pattern_roi[:,:,:,i].transpose(2, 1, 0))

## Plot projection diffraction pattern along theta axis.

In [ ]:
for i in tqdm(range(roi_num),desc="Z projection: "):
    if np.count_nonzero(colormap == i + 1) != 0:
        projection = np.average(avg_diff_pattern_roi[:,:,:,i], axis=2)
        tiff.imwrite("./projection_diffraction_pattern_" + str(i+1) + ".tif", projection.transpose(1, 0))